### From Mysql lab-sql-9

use sakila;

-- Create a table rentals_may to store the data from rental table with information for the month of May.

-- check
SELECT 
    *
FROM
    sakila.rental;

DROP TABLE IF EXISTS rentals_may;

CREATE TABLE rentals_may (
    rental_id INT UNIQUE NOT NULL,
    rental_date DATETIME,
    inventory_id INT,
    customer_id INT,
    return_date DATETIME,
    staff_id INT,
    last_update TIMESTAMP
);


-- Insert values in the table rentals_may using the table rental, filtering values only for the month of May.

INSERT INTO rentals_may 
	(SELECT * FROM rental WHERE MONTH(rental_date) = 5);
    
-- check
SELECT 
    *
FROM
    sakila.rentals_may;

-- Create a table rentals_june to store the data from rental table with information for the month of June.

DROP TABLE IF EXISTS rentals_june;

CREATE TABLE rentals_june (
    rental_id INT UNIQUE NOT NULL,
    rental_date DATETIME,
    inventory_id INT,
    customer_id INT,
    return_date DATETIME,
    staff_id INT,
    last_update TIMESTAMP
);

-- Insert values in the table rentals_june using the table rental, filtering values only for the month of June.

INSERT INTO rentals_june 
	(SELECT * FROM rental WHERE MONTH(rental_date) = 6);
    
-- check
SELECT 
    *
FROM
    sakila.rentals_june;
    
-- Check the number of rentals for each customer for May.

SELECT 
    customer_id, COUNT(*) AS n_rentals_may
FROM
    sakila.rentals_may
GROUP BY customer_id
ORDER BY n_rentals_may DESC;

-- Check the number of rentals for each customer for June.

SELECT 
    customer_id, COUNT(*) AS n_rentals_june
FROM
    sakila.rentals_june
GROUP BY customer_id
ORDER BY n_rentals_june DESC;

### Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

* Check the number of rentals for each customer for May

* Check the number of rentals for each customer for June

Hint: You can store the results from the two queries in two separate dataframes.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import getpass
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
query = 'SELECT * FROM rentals_may'
data_may = pd.read_sql_query(query, engine)
data_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [13]:
#Check the number of rentals for each customer for May
query = 'SELECT customer_id, COUNT(*) as n_rentals_may FROM rentals_may GROUP BY customer_id ORDER BY n_rentals_may DESC'
n_rentals_may = pd.read_sql_query(query, engine)
n_rentals_may.head()

,customer_id,n_rentals_may
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6


In [9]:
query = 'SELECT * FROM rentals_june'
data_june = pd.read_sql_query(query, engine)
data_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [12]:
#Check the number of rentals for each customer for June
query = 'SELECT customer_id, COUNT(*) as n_rentals_june FROM rentals_june GROUP BY customer_id ORDER BY n_rentals_june DESC'
n_rentals_june = pd.read_sql_query(query, engine)
n_rentals_june.head()

,customer_id,n_rentals_june
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9


### Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

Hint: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the merge function.

In [17]:
data = n_rentals_may.merge(n_rentals_june, left_on = 'customer_id', right_on = 'customer_id')
data.head()

,customer_id,n_rentals_may,n_rentals_june
0,197,8,8
1,506,7,5
2,109,7,5
3,269,6,3
4,239,6,5


In [22]:
may = 0
june = 0
for i in range(len(data['customer_id'])):
    if data['n_rentals_may'][i] > data['n_rentals_june'][i]:
        may += 1
    elif data['n_rentals_may'][i] < data['n_rentals_june'][i]:
        june += 1
    else:
        may = may
        june = june
        
print(f'On may {may} customers borrowed more films.');
print(f'On june {june} customers borrowed more films.');

if may > june:
    print('May wins!')
elif may < june:
    print('June wins!')
else:
    print('Both are equal.')

On may 92 customers borrowed more films.
On june 358 customers borrowed more films.
June wins!
